# Initialization
>For most applications, Python’s `__name__` variable is the correct value for this argument.
```python
from flask import Flask
app = Flask(__name__)
```
<br>

# Routes and View Functions

Easiest way to declare application route is 
```python
@app.route('/')
def index():
    return '<h1>Hello World!</h1>'
```
<br>

Application route can also be initialized as 
```python
def index():
    return '<h1>Hello World!</h1>'
app.add_url_rule('/', 'index', index)
```
<br>

For links with variable names like "www.domain.com/\<user-name>", we can create variable link with Flask as follows:
```python
@app.route('/user/<name>')
def user(name):
    return '<h1>Hello, {}!</h1>'.format(name)
```
<br>

>the route /user/\<int:id> would match only URLs that have an integer in the id dynamic segment, such as /user/123. Flask supports the types string, int, float, and path for routes. The `path` type is a special string type that can include forward slashes, unlike the string type.
<br>

# Development Web Server
Type `flask run` to terminal (tested in vscode)

or insert below code at the end
```python
if __name__ == '__main__':
    app.run()
```
to start the local server.

>While the `flask run` command makes this practice unnecessary, the `app.run()` method can still be useful on certain occasions, such as unit testing.
<br>

# Dynamic Routes
```python
@app.route('/user/<name>')
def user(name):
    return '<h1>Hello, {}!</h1>'.format(name)
```

>To test the dynamic route, make sure the server is running and then navigate to `http://localhost:5000/user/Sumanth`. The application will respond with the personalized greeting using the name dynamic argument.


# Debug Mode
>If you start your server with the app.run() method, the FLASK_APP and FLASK_DEBUG environment variables are not used. To enable debug mode programmatically, use `app.run(debug=True)`.

>Never enable debug mode on a production server. The debugger in particular allows the client to request remote code execution, so it makes your production server vulnerable to attacks. As a simple protection measure, the debugger needs to be activated with a PIN, printed to the console by the flask run command.
<br>

# Command-Line Options
```console
(venv) $ flask --help
Usage: flask [OPTIONS] COMMAND [ARGS]...

    This shell command acts as general utility script for Flask applications.

    It loads the application configured (through the FLASK_APP environment variable) and then provides commands either provided by the application or Flask itself.

    The most useful commands are the "run" and "shell" command.
    Example usage:
        $ export FLASK_APP=hello.py
        $ export FLASK_DEBUG=1
        $ flask run

Options:
    --version   Show the flask version
    --help      Show this message and exit.
Commands:
    run     Runs a development server.
    shell   Runs a shell in the app context.
```
<br>

```console
(venv) $ flask run --help
Usage: flask run [OPTIONS]

    Runs a local development server for the Flask application.

    This local server is recommended for development purposes only but it can also be used for simple intranet deployments. By default it will not support any sort of concurrency at all to simplify debugging. This can be changed with the --with-threads option which will enable basic multithreading.

    The reloader and debugger are by default enabled if the debug flag of
    Flask is enabled and disabled otherwise.

Options:
    -h, --host                          TEXT The interface to bind to.
    -p, --port                          INTEGER The port to bind to.
    --reload / --no-reload              Enable or disable the reloader. By default the reloader is active if debug is enabled.
    --debugger / --no-debugger          Enable or disable the debugger. By default the debugger is active if debug is enabled.
    --eager-loading / --lazy-loader     Enable or disable eager loading. By default eager loading is enabled if the reloader is disabled.
    --with-threads / --without-threads  Enable or disable multithreading.
    --help                              Show this message and exit.
```
<br>

>The --host argument is particularly useful because it tells the web server what network interface to listen to for connections from clients. By default, Flask’s development web server listens for connections on localhost, so only connections originating from the computer running the server are accepted. The following command makes the web server listen for connections on the public network interface, enabling other computers in the same network to connect as well:
```console
(venv) $ flask run --host 0.0.0.0
  * Serving Flask app "hello"
  * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
```
<br>

>The --reload, --no-reload, --debugger, and --no-debugger options provide a greater degree of control on top of the debug mode setting. For example, if debug mode is enabled, --no-debugger can be used to turn off the debugger, while keeping debug mode and the reloader enabled.
<br>

# The Request-Response Cycle

### Application and Request Contexts
> To avoid cluttering view functions with lots of arguments that may not always be needed, Flask uses contexts to temporarily make certain objects globally accessible. Thanks to contexts, view functions like the following one can be written:

```python
from flask import request

@app.route('/')
def index():
    user_agent = request.header.get('User-Agent')
    return '<p>Your browser is {}</p>'.format(user_agent)
```
>Note how in this view function, request is used as if it were a global variable. In reality, request cannot be a global variable; in a multithreaded server several threads can be working on different requests from different clients all at the same time, so each thread needs to see a different object in request. Contexts enable Flask to make certain variables globally accessible to a thread without interfering with the other threads.

<span style="color:red">Read and revise the Book. This section is bit difficult to comprehend.</span>

<br>

**Flask context globals**
|Variable name|Context|Description|
|-----|-----|-----|
|`current_app`|Application context|The application instance for the active application.|
|`g`|Application context|An object that the application can use for temporary storage during the handling of a request. This variable is reset with each request.|
|`request`|Request context|The request object, which encapsulates the contents of an HTTP request sent by the client.|
|`session`|Request context|The user session, a dictionary that the application can use to store values that are “remembered” between requests.|

<br>

> Flask activates (or pushes) the application and request contexts before dispatching a request to the application, and removes them after the request is handled. When the application `context` is pushed, the `current_app` and `g` variables become available to the thread. Likewise, when the request context is pushed, `request` and `session` become available as well. If any of these variables are accessed without an active application or request context, an error is generated.

<br>

### Request Dispatching
> When the application receives a request from a client, it needs to find out what view function to invoke to service it. For this task, Flask looks up the URL given in the request in the application’s URL map, which contains a mapping of URLs to the view functions that handle them. Flask builds this map using the data provided in the `app.route` decorator, or the equivalent non-decorator version, `app.add_url_rule()`.

We can see this using command `app.url_map`
```console
>>>app.url_map
Map([<Rule '/' (HEAD, OPTIONS, GET) -> index>,
<Rule '/static/<filename>' (HEAD, OPTIONS, GET) -> static>,
<Rule '/user/<name>' (HEAD, OPTIONS, GET) -> user>])
>>>
```
> The / and /user/\<name> routes were defined by the app.route decorators in the application. The /static/\<filename> route is a special route added by Flask to give access to static files.

>The (HEAD, OPTIONS, GET) elements shown in the URL map are the request methods that are handled by the routes (unless explicitly changed).

<br>

### The Request Object
**Flask request object**
|Attribute or Method|Description|
|-----|-----|
|`form`|A dictionary with all the form fields submitted with the request.|
|`args`|A dictionary with all the arguments passed in the query string of the URL.|
|`values`|A dictionary that combines the values in form and args.|
|`cookies`|A dictionary with all the cookies included in the request.|
|`headers`|A dictionary with all the HTTP headers included in the request.|
|`files`|A dictionary with all the file uploads included with the request.|
|`get_data()`|Returns the buffered data from the request body.|
|`get_json()`|Returns a Python dictionary with the parsed JSON included in the body of the request.|
|`blueprint`|The name of the Flask blueprint that is handling the request.|
|`endpoint`|The name of the Flask endpoint that is handling the request.Flask uses the name of the view function as the endpoint name for a route.|
|`method`|The HTTP request method, such as GET or POST.|
|`scheme`|The URL scheme (http or https).|
|`is_secure()`|Returns True if the request came through a secure (HTTPS) connection.|
|`host`|The host defined in the request, including the port number if given by the client.|
|`path`|The path portion of the URL.|
|`query_string`|The query string portion of the URL, as a raw binary value.|
|`full_path`|The path and query string portions of the URL.|
|`url`|The complete URL requested by the client.|
|`base_url`|Same as url, but without the query string component.|
|`remote_addr`|The IP address of the client.|
|`environ`|The raw WSGI environment dictionary for the request.|

<br>

### Request Hooks
Some codes like connection to database, authentication, etc must be setup before/after the request. We can use Request Hooks to minimize repetetion of above codes.
>Request hooks are implemented as decorators.

Flask has four hook types:
|Hook Type|Description|
|----|----|
|before_request|Registers a function to run before each request.|
|before_first_request|Registers a function to run only before the first request is handled. This can be a convenient way to add server initialization tasks.|
|after_request|Registers a function to run after each request, but only if no unhandled exceptions occurred.|
|teardown_request|Registers a function to run after each request, even if unhandled exceptions occurred.|

<br>

> A common pattern to share data between request hook functions and view functions is to use the g context global as storage.

That is to say, user login can be stored in `g.user` and is later reterieved when required.

<br>

### Responses

> HTTP protocol requires more than a string as a response to a request. A very important part of the HTTP response is the status code, which Flask by default sets to 200, the code that indicates that the request was carried out successfully.

> Flask view functions have the option of returning a response object. The `make_response()` function takes one, two, or three arguments.

For example,
```python
from flask import make_response

@app.route('/)
def index():
    response = make_response('<h1>This document carries a cookie!</h1>')
    response.set_cookie('answer', '42')
    return response
```
<br>

**Flask response object**
|Attribute or Method|Description|
|----|----|
|`status_code`|The numeric HTTP status code|
|`headers`|A dictionary-like object with all the headers that will be sent with the response|
|`set_cookie()`|Adds a cookie to the response|
|`delete_cookie()`|Removes a cookie|
|`content_length`|The length of the response body|
|`content_type`|The media type of the response body|
|`set_data()`|Sets the response body as a string or bytes value|
|`get_data()`|Gets the response body|

<br>

>There is a special type of response called a redirect. This response does not include a page document; it just gives the browser a new URL to navigate to. A redirect is typically indicated with a 302 response status code and the URL to go to given in a `Location` header. A redirect response can be generated manually with a three-value return or with a response object

> Another special response is issued with the abort() function, which is used for error handling.
>> Note that abort() does not return control back to the function because it raises an exception.

<br>

### Flask Extensions
Many functionalities like database and user authentication are not builtin in Flask. We can install suitable extensions to do these tasks.